[View in Colaboratory](https://colab.research.google.com/github/dharmesh312/CVML-COURSE/blob/master/mnistLeNetArchitecture.ipynb)

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [142]:
# get the data 
mnist = input_data.read_data_sets("MNIST_data/" ,one_hot = True, reshape = False)

test_data_images  = mnist.test.images
train_data_images = mnist.train.images
valid_data_images = mnist.validation.images

test_data_labels  = mnist.test.labels
train_data_labels = mnist.train.labels
valid_data_labels = mnist.validation.labels

print ("Train data shape :" , train_data_images.shape)
print ("test data shape :" , test_data_images.shape)
print ("Valid data shape :" , valid_data_images.shape)
print ("Shape of a image is :" , train_data_images[0].shape)

# pre processing the data
# since LeNet architectureis designed for 32 * 32 images and we have images that have dimensions of 28*28 so we need to pad the images 
test_data_images = np.pad(test_data_images , ((0,0) , (2,2) ,(2,2) , (0,0)) , 'constant' , constant_values=(0,0))
train_data_images = np.pad(train_data_images , ((0,0) , (2,2) ,(2,2) , (0,0)) , 'constant' , constant_values=(0,0))
valid_data_images = np.pad(valid_data_images , ((0,0) , (2,2) ,(2,2) , (0,0)) , 'constant' , constant_values=(0,0))


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Train data shape : (55000, 28, 28, 1)
test data shape : (10000, 28, 28, 1)
Valid data shape : (5000, 28, 28, 1)
Shape of a image is : (28, 28, 1)


In [0]:
# shuffle the training data to for normaliation
from sklearn.utils import shuffle
train_data_images , train_data_labels = shuffle(train_data_images,train_data_labels , random_state = 0) 

In [0]:
# generate convNet
from tensorflow.contrib.layers import flatten

def convNet(image):
    
    convLayer1 = {"Weights" : tf.Variable(tf.truncated_normal([5,5,1,6]  , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(6)) }
    convLayer2 = {"Weights" : tf.Variable(tf.truncated_normal([5,5,6,16] , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(16)) }
    fullyConnectedLayer1 = {"Weights" : tf.Variable(tf.truncated_normal( [400,120]  , mean = 0 ,stddev = 0.1 ))  , "Biases"  :  tf.Variable(tf.zeros(120)) }
    fullyConnectedLayer2 = {"Weights" : tf.Variable(tf.truncated_normal( [120,84]  , mean = 0 ,stddev = 0.1))  , "Biases"  :  tf.Variable(tf.zeros(84)) }
    outputLayer = {"Weights" : tf.Variable(tf.truncated_normal( [84,10] , mean = 0 ,stddev = 0.1 ))  , "Biases"  :  tf.Variable(tf.zeros(10)) }

    #     layer1 computations  ip = 32*32 op =14*14*6 
    conv1 = tf.nn.conv2d(image , convLayer1["Weights"] , strides = [1,1,1,1] , padding = 'VALID') + convLayer1['Biases']
    conv1 = tf.nn.relu(conv1)
    pool1 = tf.nn.max_pool(conv1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')

    #     layer2 computatations  ip = 14*14*6 op = 5*5*16
    conv2 = tf.nn.conv2d(pool1 , convLayer2["Weights"] , strides = [1,1,1,1] , padding = 'VALID') + convLayer2['Biases']
    conv2 = tf.nn.relu(conv2)
    pool2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # layer3 computations . Fully connected layer 400 to 120
    fc1 = flatten(pool2)
    fc1 = tf.add( tf.matmul(fc1 , fullyConnectedLayer1["Weights"]) , fullyConnectedLayer1["Biases"] )
    fc1 = tf.nn.relu(fc1)
    
    # layer4 computations . Fully connected layer 120 to 84
    fc2 = tf.add( tf.matmul(fc1 ,fullyConnectedLayer2["Weights"]) , fullyConnectedLayer2["Biases"] )
    fc2 = tf.nn.relu(fc2)
    
    # layer5 computations . output layer
    output = tf.add(tf.matmul(fc2 , outputLayer['Weights']) , outputLayer['Biases'])
    
    return output

In [0]:
# placeholders and other variables defination
epochs = 10
batch_size = 50
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder('float')
rate = 0.001

In [0]:
def train_conv_net(x):
  prediction = convNet(x)
#   print (prediction.shape , oneHotY.shape)
  cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( logits = prediction, labels = y) )
  optimizer = tf.train.AdamOptimizer(learning_rate = rate).minimize(cost)
  
  correct = tf.equal(tf.argmax(prediction, 1) , tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct ,'float'))
  
  print ("Starting training ...." )
  #taining the model
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
      train_data_image, train_data_label = shuffle(train_data_images, train_data_labels)
      loss = 0

      for start in range(0 , len(train_data_image) , batch_size):
        last = start + batch_size
        batch_x, batch_y = train_data_image[ start : last ], train_data_label[ start : last ]
        c, l = sess.run([optimizer,cost] ,feed_dict = { x:batch_x , y:batch_y })
        loss += l
      print ("Epoch:" , i , " with Loss :" , loss) 
      print ("Validation Set accuracy is :", accuracy.eval({x : valid_data_images , y : valid_data_labels}))
      
    print ("Test Set accuracy is :", accuracy.eval({x : test_data_images , y : test_data_labels}))
  
      

In [161]:
train_conv_net(x)

Starting training ....
Epoch: 0  with Loss : 140.86496805772185
Validation Set accuracy is : 0.9698
Epoch: 1  with Loss : 43.43502079509199
Validation Set accuracy is : 0.979
Epoch: 2  with Loss : 31.117777061648667
Validation Set accuracy is : 0.9836
Epoch: 3  with Loss : 23.711920015048236
Validation Set accuracy is : 0.9834
Epoch: 4  with Loss : 19.14842969318852
Validation Set accuracy is : 0.987
Epoch: 5  with Loss : 16.031015777844004
Validation Set accuracy is : 0.9856
Epoch: 6  with Loss : 13.643697672290727
Validation Set accuracy is : 0.9878
Epoch: 7  with Loss : 11.935751435696147
Validation Set accuracy is : 0.9874
Epoch: 8  with Loss : 10.69697851059027
Validation Set accuracy is : 0.9884
Epoch: 9  with Loss : 9.522179822786711
Validation Set accuracy is : 0.9874
Test Set accuracy is : 0.9846
